In [ ]:
!pip install --upgrade datasets[audio] transformers accelerate evaluate jiwer tensorboard gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 107.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 14.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 41.

In [1]:
from huggingface_hub import list_datasets, notebook_login
from datasets import load_dataset, DatasetDict

In [ ]:
# notebook_login()

In [2]:
voice_data = load_dataset("malaysia-ai/malay-conversational-speech-corpus", token=True)
voice_data

DatasetDict({
    train: Dataset({
        features: ['Y', 'id', 'gender', 'filename'],
        num_rows: 3241
    })
})

In [3]:
# rename column
voice_data_new = voice_data
voice_data_new = voice_data_new.filter(lambda x: x["id"] != "0")
voice_data_new = voice_data_new.rename_columns({"Y": "sentence", "filename": "audio"})
voice_data_new = voice_data_new.remove_columns(["id", "gender"])
voice_data_new

DatasetDict({
    train: Dataset({
        features: ['sentence', 'audio'],
        num_rows: 3149
    })
})

In [4]:
# Split the dataset into train and test
split_dataset = voice_data_new['train'].train_test_split(test_size=0.3)
split_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'audio'],
        num_rows: 2204
    })
    test: Dataset({
        features: ['sentence', 'audio'],
        num_rows: 945
    })
})

In [ ]:
split_dataset["train"][1]

{'sentence': 'golongan ah kanak-kanak yang masih bayi tu memang sangat bahaya lah sebab dia boleh menyebabkan kematian di ah di negara tersebut lah kiranya ah',
 'audio': {'path': 'A0006_S002_0_G0216-27.mp3',
  'array': array([0.00821261, 0.01074363, 0.00980355, ..., 0.01530469, 0.01590521,
         0.01459163]),
  'sampling_rate': 16000}}

In [5]:
from transformers import WhisperFeatureExtractor
from transformers import WhisperTokenizer
from transformers import WhisperProcessor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="malay", task="transcribe")
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="malay", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
input_str = split_dataset["train"][1]["sentence"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)


print(f"Input: {input_str}")
print(f"Decoded w/ special: {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal: {input_str == decoded_str}")

Input: golongan ah kanak-kanak yang masih bayi tu memang sangat bahaya lah sebab dia boleh menyebabkan kematian di ah di negara tersebut lah kiranya ah
Decoded w/ special: <|startoftranscript|><|ms|><|transcribe|><|notimestamps|>golongan ah kanak-kanak yang masih bayi tu memang sangat bahaya lah sebab dia boleh menyebabkan kematian di ah di negara tersebut lah kiranya ah<|endoftext|>
Decoded w/out special: golongan ah kanak-kanak yang masih bayi tu memang sangat bahaya lah sebab dia boleh menyebabkan kematian di ah di negara tersebut lah kiranya ah
Are equal: True


In [ ]:
print(split_dataset["train"][0])

{'sentence': 'dia punya lagi satu kalau engkau nak tukar duit Malaysia ke duit Indon sangat murah', 'audio': {'path': 'A0010_S002_0_G0626-118.mp3', 'array': array([-0.00311682, -0.00378795, -0.00344127, ..., -0.00072337,
       -0.00086411, -0.00093507]), 'sampling_rate': 16000}}


In [ ]:
# ensure that all the sampling rate is 16000
# for i in range(split_dataset.num_rows):
#     # check sampling rate unique
#     if voice_data["train"][i]["filename"]["sampling_rate"] != 16000:
#         print(voice_data["train"][i]["filename"])

In [6]:
# correcsponding audio file to 16kHz sampling rate
from datasets import Audio

common_voice = split_dataset.cast_column("audio", Audio(sampling_rate=16000))
common_voice

DatasetDict({
    train: Dataset({
        features: ['sentence', 'audio'],
        num_rows: 2204
    })
    test: Dataset({
        features: ['sentence', 'audio'],
        num_rows: 945
    })
})

In [ ]:
print(split_dataset["train"][0])
print(common_voice["train"][0])

{'sentence': 'dia punya lagi satu kalau engkau nak tukar duit Malaysia ke duit Indon sangat murah', 'audio': {'path': 'A0010_S002_0_G0626-118.mp3', 'array': array([-0.00311682, -0.00378795, -0.00344127, ..., -0.00072337,
       -0.00086411, -0.00093507]), 'sampling_rate': 16000}}
{'sentence': 'dia punya lagi satu kalau engkau nak tukar duit Malaysia ke duit Indon sangat murah', 'audio': {'path': 'A0010_S002_0_G0626-118.mp3', 'array': array([-0.00311682, -0.00378795, -0.00344127, ..., -0.00072337,
       -0.00086411, -0.00093507]), 'sampling_rate': 16000}}


In [8]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]
    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    batch["input_length"] = len(audio["array"]) / audio["sampling_rate"]
    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

common_voice_new = common_voice.map(prepare_dataset,remove_columns=common_voice.column_names["train"])
common_voice_new

Map:   0%|          | 0/2204 [00:00<?, ? examples/s]

Map:   0%|          | 0/945 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_features', 'input_length', 'labels'],
        num_rows: 2204
    })
    test: Dataset({
        features: ['input_features', 'input_length', 'labels'],
        num_rows: 945
    })
})

In [11]:
common_voice_new["train"].features

{'input_features': Sequence(feature=Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None), length=-1, id=None),
 'input_length': Value(dtype='float64', id=None),
 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}

# Load a Pre-Trained Checkpoint

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

In [ ]:
model.generation_config.language = "malay"
model.generation_config.task = "transcribe"
model.generation_config.forced_decoder_ids = None
model.generation_config.suppress_tokens = []

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id)

In [ ]:
import evaluate

metric = evaluate.load("wer")

In [ ]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-malay-ft-test-3",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=1000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=100,
    eval_steps=100,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice_new["train"],
    eval_dataset=common_voice_new["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
processor.save_pretrained(training_args.output_dir)

[]

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

!mkdir -p /content/drive/MyDrive/whisper_model_3

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
100,0.727500,0.677592,38.911115
200,0.521000,0.565486,36.615063
300,0.312800,0.525294,29.796486
400,0.296400,0.500519,35.223517
500,0.163100,0.508256,36.284571
600,0.073100,0.532225,38.441468
700,0.054800,0.519905,27.274309
800,0.028900,0.533013,27.691772
900,0.013100,0.548259,26.909028
1000,0.007100,0.545344,27.169943


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint

TrainOutput(global_step=1000, training_loss=0.2528136772066355, metrics={'train_runtime': 8606.2007, 'train_samples_per_second': 1.859, 'train_steps_per_second': 0.116, 'total_flos': 4.60928601391104e+18, 'train_loss': 0.2528136772066355, 'epoch': 7.246376811594203})

In [ ]:
# Copy the entire output directory to Google Drive
!cp -r ./whisper-small-malay-ft-test-3 /content/drive/MyDrive/whisper_model_3

cp: error writing '/content/drive/MyDrive/whisper_model_3/whisper-small-malay-ft-test-3/checkpoint-600/model.safetensors': No space left on device
cp: error writing '/content/drive/MyDrive/whisper_model_3/whisper-small-malay-ft-test-3/checkpoint-600/preprocessor_config.json': No space left on device
cp: error writing '/content/drive/MyDrive/whisper_model_3/whisper-small-malay-ft-test-3/checkpoint-600/training_args.bin': No space left on device
cp: error writing '/content/drive/MyDrive/whisper_model_3/whisper-small-malay-ft-test-3/checkpoint-600/optimizer.pt': No space left on device
cp: error writing '/content/drive/MyDrive/whisper_model_3/whisper-small-malay-ft-test-3/checkpoint-600/scheduler.pt': No space left on device
cp: error writing '/content/drive/MyDrive/whisper_model_3/whisper-small-malay-ft-test-3/checkpoint-600/rng_state.pth': No space left on device
cp: error writing '/content/drive/MyDrive/whisper_model_3/whisper-small-malay-ft-test-3/checkpoint-600/trainer_state.json': N

In [ ]:
model = WhisperForConditionalGeneration.from_pretrained('./whisper-small-malay-ft-test-3')
tokenizer = WhisperTokenizer.from_pretrained('./whisper-small-malay-ft-test-3')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
